In [75]:
import requests 
import bs4
import pandas as pd
import numpy as np

In [76]:
EDIBLE_FRUITS_URL = 'https://en.wikipedia.org/wiki/List_of_culinary_fruits'

In [77]:
response = requests.get(EDIBLE_FRUITS_URL)
if response.status_code == 200:
    print ("Get Successfull")
else:
    print ("Get Failed")

Get Successfull


In [78]:
resp_soap = bs4.BeautifulSoup(response.text)

In [79]:
resp_soap.title

<title>List of culinary fruits - Wikipedia</title>

###### Header of the first table

In [80]:
resp_soap.table.tbody.tr

<tr>
<th><a href="/wiki/Common_name" title="Common name">Common name</a>
</th>
<th><a href="/wiki/Binomial_nomenclature" title="Binomial nomenclature">Species name</a>
</th>
<th><a href="/wiki/Cultivar" title="Cultivar">Cultivar list</a>
</th></tr>

In [81]:
def add_to_dictionary(table_dict):
    
    if (row_list[0]) in table_dict:
        table_dict[row_list[0]][1] = pos_dict[row_list[0]][1] + ", " + row_list[2]
    else:
        table_dict[row_list[0]] = [row_list[1],row_list[2]]

In [82]:
def process_row(row,ncolumns=0):
    
    row_values = []
    col_count = 0
    for column in row:
        
        if isinstance(column,bs4.element.Tag): # Avoid processing NavigableString - Mostly newline char      
            col_count += 1
            row_values.append(column.get_text(strip=True))
            
        if ncolumns > 0 and col_count >= ncolumns:
            break
            
    return row_values

In [83]:
def process_table(soap_msg_table,ncolumns=0):
    
    row_list = []
    for row in soap_msg_table:
        
        if isinstance(row,bs4.element.Tag): # Avoid processing NavigableString - Mostly newline char
            row_list.append(process_row(row,ncolumns))
            
    #print (row_list)
    return row_list

In [84]:
all_common_spices = pd.DataFrame()
for table in resp_soap.find_all("table"):
    common_spices = pd.DataFrame(process_table(table.tbody,ncolumns=3))
    header = common_spices.loc[0]
    common_spices = common_spices[1:]
    all_common_spices = all_common_spices.append(common_spices)
all_common_spices.columns = header

In [85]:
all_common_spices.head()

,Common name,Species name,Cultivar list
1,Apple,Malus pumila,Apple cultivars
2,Chinese quince,Pseudocydonia sinensis,
3,Chokeberry,Aronia melanocarpa,
4,Cocky apple,Planchonia careya,
5,Eastern mayhaw,Crataegus aestivalis,


In [86]:
all_common_spices.shape

(422, 3)

In [87]:
all_common_spices.loc[all_common_spices['Common name'] == 'Saguaro fruit']

,Common name,Species name,Cultivar list
7,Saguaro fruit,Carnegiea gigantea,
